# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [51]:
problem = '''A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased.'''

## 2. Ask for parameters

In [70]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [71]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Decision Variables:

*  **Xa:** Units purchased from Vendor A (in thousands)
*  **Xb1:** Units purchased from Vendor B under the first pricing tier (22,000,000-70,000,000 units) (in thousands)
*  **Xb2:** Units purchased from Vendor B under the second pricing tier (70,000,001-100,000,000 units) (in thousands)
*  **Xb3:** Units purchased from Vendor B under the third pricing tier (100,000,001-150,000,000 units) (in thousands)
*  **Xb4:** Units purchased from Vendor B under the fourth pricing tier (150,000,001-160,000,000 units) (in thousands)
*  **Xc:** Units purchased from Vendor C (in thousands)
*  **Xd:** Units purchased from Vendor D (in thousands)
*  **Xe1:** Units purchased from Vendor E under the first pricing tier (0-42,000,000 units) (in thousands)
*  **Xe2:** Units purchased from Vendor E under the second pricing tier (42,000,001-77,000,000 units) (in thousands) 

## Binary Variables:

* **Yb1:** 1 if any quantity is purchased from Vendor B within the first pricing tier, 0 otherwise.
* **Yb2:** 1 if any quantity is purchased from Vendor B within the second pricing tier, 0 otherwise.
* **Yb3:** 1 if any quantity is purchased from Vendor B within the third pricing tier, 0 otherwise.
* **Yb4:** 1 if any quantity is purchased from Vendor B within the fourth pricing tier, 0 otherwise.
* **Ye1:** 1 if any quantity is purchased from Vendor E within the first pricing tier, 0 otherwise.
* **Ye2:** 1 if any quantity is purchased from Vendor E within the second pricing tier, 0 otherwise. 


# 2. Ask for objective

In [72]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [73]:
Markdown(response2.text)

Minimize: 

```
Total Cost = (3855.34 * Xa) + (61.150 * Xa) + (125804.84 * Yb1) + (68.099 * Xb1) + (269304.84 * Yb2) + (66.049 * Xb2) + (464304.84 * Yb3) + (64.099 * Xb3) + (761304.84 * Yb4) + (62.119 * Xb4) + (13456 * Xc) + (62.019 * Xc) + (6583.98 * Xd) + (72.488 * Xd) + (70.150 * Xe1) + (84000 * Ye2) +  Σ(from i=0 to Xe2-1) [ (68.150 - (0.05 * i)) ]
``` 


# 3. Ask for constraints

In [74]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [75]:
Markdown(response3.text)

## Constraints:

**Demand Constraint:**

* Xa + Xb1 + Xb2 + Xb3 + Xb4 + Xc + Xd + Xe1 + Xe2 = 239,600.48 

**Supplier Capacity Constraints:**

* Xa <= 33,000
* Xb1 + Xb2 + Xb3 + Xb4 <= 160,000
* Xc <= 165,600
* Xd <= 12,000
* Xe1 + Xe2 <= 77,000

**Vendor B Minimum Purchase Constraints:**

* Xb1 >= 22,000 * Yb1 
* Xb1 <= 70,000 * Yb1
* Xb2 >= 70,001 * Yb2
* Xb2 <= 100,000 * Yb2
* Xb3 >= 100,001 * Yb3
* Xb3 <= 150,000 * Yb3
* Xb4 >= 150,001 * Yb4
* Xb4 <= 160,000 * Yb4

**Vendor E Pricing Tier Constraints:**

* Xe1 <= 42,000 * Ye1
* Xe2 >= 42,001 * Ye2
* Xe2 <= 77,000 * Ye2

**Binary Variable Constraints:**

* Yb1 + Yb2 + Yb3 + Yb4 <= 1 
* Ye1 + Ye2 <= 1

**Non-Negativity Constraints:**

* Xa, Xb1, Xb2, Xb3, Xb4, Xc, Xd, Xe1, Xe2 >= 0

**Binary Variable Definitions:**

* Yb1, Yb2, Yb3, Yb4, Ye1, Ye2 ∈ {0, 1} 


## 3. Generate the pyomo code

In [76]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [77]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample data -  Replace with actual data if available
demand = 239600.48

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.Xa = pyo.Var(bounds=(0, None))
model.Xb1 = pyo.Var(bounds=(0, None))
model.Xb2 = pyo.Var(bounds=(0, None))
model.Xb3 = pyo.Var(bounds=(0, None))
model.Xb4 = pyo.Var(bounds=(0, None))
model.Xc = pyo.Var(bounds=(0, None))
model.Xd = pyo.Var(bounds=(0, None))
model.Xe1 = pyo.Var(bounds=(0, None))
model.Xe2 = pyo.Var(bounds=(0, None))

# Binary Variables
model.Yb1 = pyo.Var(within=pyo.Binary)
model.Yb2 = pyo.Var(within=pyo.Binary)
model.Yb3 = pyo.Var(within=pyo.Binary)
model.Yb4 = pyo.Var(within=pyo.Binary)
model.Ye1 = pyo.Var(within=pyo.Binary)
model.Ye2 = pyo.Var(within=pyo.Binary)

# Objective function
def objective_rule(model):
    return (3855.34 * model.Xa) + (61.150 * model.Xa) + (125804.84 * model.Yb1) + (68.099 * model.Xb1) + (269304.84 * model.Yb2) + (66.049 * model.Xb2) + (464304.84 * model.Yb3) + (64.099 * model.Xb3) + (761304.84 * model.Yb4) + (62.119 * model.Xb4) + (13456 * model.Xc) + (62.019 * model.Xc) + (6583.98 * model.Xd) + (72.488 * model.Xd) + (70.150 * model.Xe1) + (84000 * model.Ye2) +  sum( (68.150 - (0.05 * i)) for i in range(1, int(model.Xe2.value)+1))

model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.Xa + model.Xb1 + model.Xb2 + model.Xb3 + model.Xb4 + model.Xc + model.Xd + model.Xe1 + model.Xe2 == demand)

model.supplier_a_constraint = pyo.Constraint(expr=model.Xa <= 33000)
model.supplier_b_constraint = pyo.Constraint(expr=model.Xb1 + model.Xb2 + model.Xb3 + model.Xb4 <= 160000)
model.supplier_c_constraint = pyo.Constraint(expr=model.Xc <= 165600)
model.supplier_d_constraint = pyo.Constraint(expr=model.Xd <= 12000)
model.supplier_e_constraint = pyo.Constraint(expr=model.Xe1 + model.Xe2 <= 77000)

model.vendor_b_min_constraint1 = pyo.Constraint(expr=model.Xb1 >= 22000 * model.Yb1)
model.vendor_b_max_constraint1 = pyo.Constraint(expr=model.Xb1 <= 70000 * model.Yb1)
model.vendor_b_min_constraint2 = pyo.Constraint(expr=model.Xb2 >= 70001 * model.Yb2)
model.vendor_b_max_constraint2 = pyo.Constraint(expr=model.Xb2 <= 100000 * model.Yb2)
model.vendor_b_min_constraint3 = pyo.Constraint(expr=model.Xb3 >= 100001 * model.Yb3)
model.vendor_b_max_constraint3 = pyo.Constraint(expr=model.Xb3 <= 150000 * model.Yb3)
model.vendor_b_min_constraint4 = pyo.Constraint(expr=model.Xb4 >= 150001 * model.Yb4)
model.vendor_b_max_constraint4 = pyo.Constraint(expr=model.Xb4 <= 160000 * model.Yb4)

model.vendor_e_constraint1 = pyo.Constraint(expr=model.Xe1 <= 42000 * model.Ye1)
model.vendor_e_min_constraint2 = pyo.Constraint(expr=model.Xe2 >= 42001 * model.Ye2)
model.vendor_e_max_constraint2 = pyo.Constraint(expr=model.Xe2 <= 77000 * model.Ye2)

model.binary_constraint_b = pyo.Constraint(expr=model.Yb1 + model.Yb2 + model.Yb3 + model.Yb4 <= 1)
model.binary_constraint_e = pyo.Constraint(expr=model.Ye1 + model.Ye2 <= 1)

# Solve the model
solver = pyo.SolverFactory('cbc') # Replace 'cbc' with another solver if desired
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal Solution:")
print("Xa =", model.Xa.value)
print("Xb1 =", model.Xb1.value)
print("Xb2 =", model.Xb2.value)
print("Xb3 =", model.Xb3.value)
print("Xb4 =", model.Xb4.value)
print("Xc =", model.Xc.value)
print("Xd =", model.Xd.value)
print("Xe1 =", model.Xe1.value)
print("Xe2 =", model.Xe2.value)
print("Total Cost =", model.total_cost.value)
```


## 4. Input problem data and try running the generated code

In [78]:
import pyomo.environ as pyo

# Sample data -  Replace with actual data if available
demand = 239600.48

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.Xa = pyo.Var(bounds=(0, None))
model.Xb1 = pyo.Var(bounds=(0, None))
model.Xb2 = pyo.Var(bounds=(0, None))
model.Xb3 = pyo.Var(bounds=(0, None))
model.Xb4 = pyo.Var(bounds=(0, None))
model.Xc = pyo.Var(bounds=(0, None))
model.Xd = pyo.Var(bounds=(0, None))
model.Xe1 = pyo.Var(bounds=(0, None))
model.Xe2 = pyo.Var(bounds=(0, None))

# Binary Variables
model.Yb1 = pyo.Var(within=pyo.Binary)
model.Yb2 = pyo.Var(within=pyo.Binary)
model.Yb3 = pyo.Var(within=pyo.Binary)
model.Yb4 = pyo.Var(within=pyo.Binary)
model.Ye1 = pyo.Var(within=pyo.Binary)
model.Ye2 = pyo.Var(within=pyo.Binary)

# Objective function
def objective_rule(model):
    return (3855.34 * model.Xa) + (61.150 * model.Xa) + (125804.84 * model.Yb1) + (68.099 * model.Xb1) + (269304.84 * model.Yb2) + (66.049 * model.Xb2) + (464304.84 * model.Yb3) + (64.099 * model.Xb3) + (761304.84 * model.Yb4) + (62.119 * model.Xb4) + (13456 * model.Xc) + (62.019 * model.Xc) + (6583.98 * model.Xd) + (72.488 * model.Xd) + (70.150 * model.Xe1) + (84000 * model.Ye2) +  sum( (68.150 - (0.05 * i)) for i in range(1, int(model.Xe2.value)+1))

model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.Xa + model.Xb1 + model.Xb2 + model.Xb3 + model.Xb4 + model.Xc + model.Xd + model.Xe1 + model.Xe2 == demand)

model.supplier_a_constraint = pyo.Constraint(expr=model.Xa <= 33000)
model.supplier_b_constraint = pyo.Constraint(expr=model.Xb1 + model.Xb2 + model.Xb3 + model.Xb4 <= 160000)
model.supplier_c_constraint = pyo.Constraint(expr=model.Xc <= 165600)
model.supplier_d_constraint = pyo.Constraint(expr=model.Xd <= 12000)
model.supplier_e_constraint = pyo.Constraint(expr=model.Xe1 + model.Xe2 <= 77000)

model.vendor_b_min_constraint1 = pyo.Constraint(expr=model.Xb1 >= 22000 * model.Yb1)
model.vendor_b_max_constraint1 = pyo.Constraint(expr=model.Xb1 <= 70000 * model.Yb1)
model.vendor_b_min_constraint2 = pyo.Constraint(expr=model.Xb2 >= 70001 * model.Yb2)
model.vendor_b_max_constraint2 = pyo.Constraint(expr=model.Xb2 <= 100000 * model.Yb2)
model.vendor_b_min_constraint3 = pyo.Constraint(expr=model.Xb3 >= 100001 * model.Yb3)
model.vendor_b_max_constraint3 = pyo.Constraint(expr=model.Xb3 <= 150000 * model.Yb3)
model.vendor_b_min_constraint4 = pyo.Constraint(expr=model.Xb4 >= 150001 * model.Yb4)
model.vendor_b_max_constraint4 = pyo.Constraint(expr=model.Xb4 <= 160000 * model.Yb4)

model.vendor_e_constraint1 = pyo.Constraint(expr=model.Xe1 <= 42000 * model.Ye1)
model.vendor_e_min_constraint2 = pyo.Constraint(expr=model.Xe2 >= 42001 * model.Ye2)
model.vendor_e_max_constraint2 = pyo.Constraint(expr=model.Xe2 <= 77000 * model.Ye2)

model.binary_constraint_b = pyo.Constraint(expr=model.Yb1 + model.Yb2 + model.Yb3 + model.Yb4 <= 1)
model.binary_constraint_e = pyo.Constraint(expr=model.Ye1 + model.Ye2 <= 1)

# Solve the model
solver = pyo.SolverFactory('glpk') # Replace 'cbc' with another solver if desired
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal Solution:")
print("Xa =", model.Xa.value)
print("Xb1 =", model.Xb1.value)
print("Xb2 =", model.Xb2.value)
print("Xb3 =", model.Xb3.value)
print("Xb4 =", model.Xb4.value)
print("Xc =", model.Xc.value)
print("Xd =", model.Xd.value)
print("Xe1 =", model.Xe1.value)
print("Xe2 =", model.Xe2.value)
print("Total Cost =", model.total_cost.value)

ERROR: Rule failed when generating expression for Objective total_cost with
index None: TypeError: int() argument must be a string, a bytes-like object or
a real number, not 'NoneType'
ERROR: Constructing component 'total_cost' from data=None failed:
        TypeError: int() argument must be a string, a bytes-like object or a
        real number, not 'NoneType'


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

## 5. Correct the code to verify model viability (optional)

In [86]:
import pyomo.environ as pyo

# Sample data -  Replace with actual data if available
demand = 239600.48

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.Xa = pyo.Var(bounds=(0, None))
model.Xb1 = pyo.Var(bounds=(0, None))
model.Xb2 = pyo.Var(bounds=(0, None))
model.Xb3 = pyo.Var(bounds=(0, None))
model.Xb4 = pyo.Var(bounds=(0, None))
model.Xc = pyo.Var(bounds=(0, None))
model.Xd = pyo.Var(bounds=(0, None))
model.Xe1 = pyo.Var(bounds=(0, None))
model.Xe2 = pyo.Var(bounds=(0, None))

# Binary Variables
model.Yb1 = pyo.Var(within=pyo.Binary)
model.Yb2 = pyo.Var(within=pyo.Binary)
model.Yb3 = pyo.Var(within=pyo.Binary)
model.Yb4 = pyo.Var(within=pyo.Binary)
model.Ye1 = pyo.Var(within=pyo.Binary)
model.Ye2 = pyo.Var(within=pyo.Binary)

# Objective function
def objective_rule(model):
    return (3855.34 * model.Xa) + (61.150 * model.Xa) 
+ (125804.84 * model.Yb1) + (68.099 * model.Xb1) + (269304.84 * model.Yb2) 
+ (66.049 * model.Xb2) + (464304.84 * model.Yb3) + (64.099 * model.Xb3) 
+ (761304.84 * model.Yb4) + (62.119 * model.Xb4) + (13456 * model.Xc) 
+ (62.019 * model.Xc) + (6583.98 * model.Xd) + (72.488 * model.Xd) 
+ (70.150 * model.Xe1) + (84000 * model.Ye2) 
+  sum( (68.150 - (0.05 * i)) for i in range(1, (model.Xe2.value)+1))

model.total_cost = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

# Constraints
model.demand_constraint = pyo.Constraint(expr=model.Xa + model.Xb1 + model.Xb2 + model.Xb3 + model.Xb4 + model.Xc + model.Xd + model.Xe1 + model.Xe2 == demand)

model.supplier_a_constraint = pyo.Constraint(expr=model.Xa <= 33000)
model.supplier_b_constraint = pyo.Constraint(expr=model.Xb1 + model.Xb2 + model.Xb3 + model.Xb4 <= 160000)
model.supplier_c_constraint = pyo.Constraint(expr=model.Xc <= 165600)
model.supplier_d_constraint = pyo.Constraint(expr=model.Xd <= 12000)
model.supplier_e_constraint = pyo.Constraint(expr=model.Xe1 + model.Xe2 <= 77000)

model.vendor_b_min_constraint1 = pyo.Constraint(expr=model.Xb1 >= 22000 * model.Yb1)
model.vendor_b_max_constraint1 = pyo.Constraint(expr=model.Xb1 <= 70000 * model.Yb1)
model.vendor_b_min_constraint2 = pyo.Constraint(expr=model.Xb2 >= 70001 * model.Yb2)
model.vendor_b_max_constraint2 = pyo.Constraint(expr=model.Xb2 <= 100000 * model.Yb2)
model.vendor_b_min_constraint3 = pyo.Constraint(expr=model.Xb3 >= 100001 * model.Yb3)
model.vendor_b_max_constraint3 = pyo.Constraint(expr=model.Xb3 <= 150000 * model.Yb3)
model.vendor_b_min_constraint4 = pyo.Constraint(expr=model.Xb4 >= 150001 * model.Yb4)
model.vendor_b_max_constraint4 = pyo.Constraint(expr=model.Xb4 <= 160000 * model.Yb4)

model.vendor_e_constraint1 = pyo.Constraint(expr=model.Xe1 <= 42000 * model.Ye1)
model.vendor_e_min_constraint2 = pyo.Constraint(expr=model.Xe2 >= 42001 * model.Ye2)
model.vendor_e_max_constraint2 = pyo.Constraint(expr=model.Xe2 <= 77000 * model.Ye2)

model.binary_constraint_b = pyo.Constraint(expr=model.Yb1 + model.Yb2 + model.Yb3 + model.Yb4 <= 1)
model.binary_constraint_e = pyo.Constraint(expr=model.Ye1 + model.Ye2 <= 1)

# Solve the model
solver = pyo.SolverFactory('glpk') # Replace 'cbc' with another solver if desired
results = solver.solve(model)

# Print the results
print("Status:", results.solver.status)
print("Termination condition:",results.solver.termination_condition)
print("Optimal Solution:")
print("Xa =", model.Xa.value)
print("Xb1 =", model.Xb1.value)
print("Xb2 =", model.Xb2.value)
print("Xb3 =", model.Xb3.value)
print("Xb4 =", model.Xb4.value)
print("Xc =", model.Xc.value)
print("Xd =", model.Xd.value)
print("Xe1 =", model.Xe1.value)
print("Xe2 =", model.Xe2.value)
print("Total Cost =", model.total_cost.value)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

## 6. Print the responses

In [87]:
print(response.text)

## Decision Variables:

*  **Xa:** Units purchased from Vendor A (in thousands)
*  **Xb1:** Units purchased from Vendor B under the first pricing tier (22,000,000-70,000,000 units) (in thousands)
*  **Xb2:** Units purchased from Vendor B under the second pricing tier (70,000,001-100,000,000 units) (in thousands)
*  **Xb3:** Units purchased from Vendor B under the third pricing tier (100,000,001-150,000,000 units) (in thousands)
*  **Xb4:** Units purchased from Vendor B under the fourth pricing tier (150,000,001-160,000,000 units) (in thousands)
*  **Xc:** Units purchased from Vendor C (in thousands)
*  **Xd:** Units purchased from Vendor D (in thousands)
*  **Xe1:** Units purchased from Vendor E under the first pricing tier (0-42,000,000 units) (in thousands)
*  **Xe2:** Units purchased from Vendor E under the second pricing tier (42,000,001-77,000,000 units) (in thousands) 

## Binary Variables:

* **Yb1:** 1 if any quantity is purchased from Vendor B within the first pricing tier, 0 

In [88]:
print(response2.text)

Minimize: 

```
Total Cost = (3855.34 * Xa) + (61.150 * Xa) + (125804.84 * Yb1) + (68.099 * Xb1) + (269304.84 * Yb2) + (66.049 * Xb2) + (464304.84 * Yb3) + (64.099 * Xb3) + (761304.84 * Yb4) + (62.119 * Xb4) + (13456 * Xc) + (62.019 * Xc) + (6583.98 * Xd) + (72.488 * Xd) + (70.150 * Xe1) + (84000 * Ye2) +  Σ(from i=0 to Xe2-1) [ (68.150 - (0.05 * i)) ]
``` 



In [89]:
print(response3.text)

## Constraints:

**Demand Constraint:**

* Xa + Xb1 + Xb2 + Xb3 + Xb4 + Xc + Xd + Xe1 + Xe2 = 239,600.48 

**Supplier Capacity Constraints:**

* Xa <= 33,000
* Xb1 + Xb2 + Xb3 + Xb4 <= 160,000
* Xc <= 165,600
* Xd <= 12,000
* Xe1 + Xe2 <= 77,000

**Vendor B Minimum Purchase Constraints:**

* Xb1 >= 22,000 * Yb1 
* Xb1 <= 70,000 * Yb1
* Xb2 >= 70,001 * Yb2
* Xb2 <= 100,000 * Yb2
* Xb3 >= 100,001 * Yb3
* Xb3 <= 150,000 * Yb3
* Xb4 >= 150,001 * Yb4
* Xb4 <= 160,000 * Yb4

**Vendor E Pricing Tier Constraints:**

* Xe1 <= 42,000 * Ye1
* Xe2 >= 42,001 * Ye2
* Xe2 <= 77,000 * Ye2

**Binary Variable Constraints:**

* Yb1 + Yb2 + Yb3 + Yb4 <= 1 
* Ye1 + Ye2 <= 1

**Non-Negativity Constraints:**

* Xa, Xb1, Xb2, Xb3, Xb4, Xc, Xd, Xe1, Xe2 >= 0

**Binary Variable Definitions:**

* Yb1, Yb2, Yb3, Yb4, Ye1, Ye2 ∈ {0, 1} 



In [90]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample data -  Replace with actual data if available
demand = 239600.48

# Model
model = pyo.ConcreteModel()

# Decision Variables
model.Xa = pyo.Var(bounds=(0, None))
model.Xb1 = pyo.Var(bounds=(0, None))
model.Xb2 = pyo.Var(bounds=(0, None))
model.Xb3 = pyo.Var(bounds=(0, None))
model.Xb4 = pyo.Var(bounds=(0, None))
model.Xc = pyo.Var(bounds=(0, None))
model.Xd = pyo.Var(bounds=(0, None))
model.Xe1 = pyo.Var(bounds=(0, None))
model.Xe2 = pyo.Var(bounds=(0, None))

# Binary Variables
model.Yb1 = pyo.Var(within=pyo.Binary)
model.Yb2 = pyo.Var(within=pyo.Binary)
model.Yb3 = pyo.Var(within=pyo.Binary)
model.Yb4 = pyo.Var(within=pyo.Binary)
model.Ye1 = pyo.Var(within=pyo.Binary)
model.Ye2 = pyo.Var(within=pyo.Binary)

# Objective function
def objective_rule(model):
    return (3855.34 * model.Xa) + (61.150 * model.Xa) + (125804.84 * model.Yb1) + (68.099 * model.Xb1) + (269304.84 * model.Yb2) + (66.049 * model.Xb2) + (464304.84 * model.Yb3) + (64.09